<a href="https://colab.research.google.com/github/owoicho/FSDP/blob/main/transformer_wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: install pytorch

!pip install torch torchvision torchaudio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [2]:
# prompt: import torct, import partial, Verify we have FSDP activation support ready by importing

import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.distributed.fsdp import FullyShardedDataParallel as FSDP
from torch.distributed.fsdp.fully_sharded_data_parallel import (
    CPUOffload,
    BackwardPrefetch,
)
from functools import partial

# Verify we have FSDP activation support ready by importing
# the activation checkpointing-related functions.
from torch.distributed.fsdp.wrap import (
    size_based_auto_wrap_policy,
    enable_wrap,
)
from torch.distributed.algorithms._checkpoint.checkpoint_wrapper import (
    checkpoint_wrapper,
    CheckpointImpl,
    apply_activation_checkpointing_wrapper,
    apply_activation_checkpointing_for_module,
)

# Example usage (replace with your actual model and training loop)

# Dummy model
class MyModel(torch.nn.Module):
  def __init__(self):
      super().__init__()
      self.linear = torch.nn.Linear(10, 10)

  def forward(self, x):
      return self.linear(x)

def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'

    dist.init_process_group("nccl", rank=rank, world_size=world_size)

def cleanup():
    dist.destroy_process_group()


def demo_basic(rank, world_size):
    print(f"Running basic DDP example on rank {rank}.")
    setup(rank, world_size)

    model = MyModel()

    # Wrap the model with FSDP
    fsdp_model = FSDP(model)

    # Example forward pass (replace with your actual training code)
    input_tensor = torch.randn(1,10)
    output = fsdp_model(input_tensor)

    print(f"Output shape on rank {rank}: {output.shape}")

    cleanup()

if __name__ == "__main__":
    world_size = 2 # Replace with the number of processes you want to use
    mp.spawn(demo_basic, args=(world_size,), nprocs=world_size, join=True)


W0415 22:04:30.387000 475 torch/multiprocessing/spawn.py:169] Terminating process 22045 via signal SIGTERM


ProcessExitedException: process 0 terminated with exit code 1

In [ ]:
# prompt: import torch

import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.distributed.fsdp import FullyShardedDataParallel as FSDP
from torch.distributed.fsdp.fully_sharded_data_parallel import (
    CPUOffload,
    BackwardPrefetch,
)

from functools import partial
from torch.distributed.fsdp.wrap import (
    size_based_auto_wrap_policy,
    enable_wrap,
)
from torch.distributed.algorithms._checkpoint.checkpoint_wrapper import (
    checkpoint_wrapper,
    CheckpointImpl,
    apply_activation_checkpointing_wrapper,
    apply_activation_checkpointing_for_module,
)


# Example usage (replace with your actual model and training loop)

# Dummy model
class MyModel(torch.nn.Module):
  def __init__(self):
      super().__init__()
      self.linear = torch.nn.Linear(10, 10)

  def forward(self, x):
      return self.linear(x)

def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'

    dist.init_process_group("nccl", rank=rank, world_size=world_size)

def cleanup():
    dist.destroy_process_group()


def demo_basic(rank, world_size):
    print(f"Running basic DDP example on rank {rank}.")
    setup(rank, world_size)

    model = MyModel()

    # Wrap the model with FSDP
    fsdp_model = FSDP(model)

    # Example forward pass (replace with your actual training code)
    input_tensor = torch.randn(1,10)
    output = fsdp_model(input_tensor)

    print(f"Output shape on rank {rank}: {output.shape}")

    cleanup()

if __name__ == "__main__":
    world_size = 2 # Replace with the number of processes you want to use
    mp.spawn(demo_basic, args=(world_size,), nprocs=world_size, join=True)
